In [1]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [2]:
n_groups_per_col = 30
ind_categorical_data_train = pd.read_feather("../data/ind_categorical_data_train.feather")
ind_categorical_labels_train = pd.read_feather("../data/ind_categorical_labels_train.feather")

In [3]:
train_data = ind_categorical_data_train.to_numpy()
train_label_close = to_categorical(ind_categorical_labels_train.close.to_numpy(), n_groups_per_col)

In [4]:
def fit_ind_categorical_model(train_data, train_labels):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(n_groups_per_col, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(train_data, train_labels, validation_split=0.2, batch_size=512, epochs=20, verbose=1)
    
    return (history, model)

In [5]:
# low
train_label_low = to_categorical(ind_categorical_labels_train.low.to_numpy(), n_groups_per_col)
(indipendent_categorical_low_history, indipendent_categorical_low_model) = fit_ind_categorical_model(train_data, train_label_low)
feather.write_dataframe(pd.DataFrame.from_dict(indipendent_categorical_low_history.history), "../data/indipendent_categorical_low_history.feather")
indipendent_categorical_low_model.save("../data/indipendent_categorical_low_model.hdf5")

# high
train_label_high = to_categorical(ind_categorical_labels_train.high.to_numpy(), n_groups_per_col)
(indipendent_categorical_high_history, indipendent_categorical_high_model) = fit_ind_categorical_model(train_data, train_label_high)
feather.write_dataframe(pd.DataFrame.from_dict(indipendent_categorical_high_history.history), "../data/indipendent_categorical_high_history.feather")
indipendent_categorical_high_model.save("../data/indipendent_categorical_high_model.hdf5")

# close
train_label_close = to_categorical(ind_categorical_labels_train.close.to_numpy(), n_groups_per_col)
(indipendent_categorical_close_history, indipendent_categorical_close_model) = fit_ind_categorical_model(train_data, train_label_close)
feather.write_dataframe(pd.DataFrame.from_dict(indipendent_categorical_close_history.history), "../data/indipendent_categorical_close_history.feather")
indipendent_categorical_close_model.save("../data/indipendent_categorical_close_model.hdf5")


Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
4342137/4342137 [==============================] - 32s 7us/step - loss: 3.2727 - accuracy: 0.0806 - val_loss: 3.1234 - val_accuracy: 0.1051
Epoch 2/20
4342137/4342137 [==============================] - 33s 8us/step - loss: 3.1757 - accuracy: 0.0899 - val_loss: 3.0821 - val_accuracy: 0.1033
Epoch 3/20
4342137/4342137 [==============================] - 33s 8us/step - loss: 3.1217 - accuracy: 0.0968 - val_loss: 3.2300 - val_accuracy: 0.0718
Epoch 4/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.1069 - accuracy: 0.0986 - val_loss: 3.1174 - val_accuracy: 0.0997
Epoch 5/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.1490 - accuracy: 0.0938 - val_loss: 3.2244 - val_accuracy: 0.0769
Epoch 6/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.0996 - accuracy: 0.0992 - val_loss: 3.0555 - val_accuracy: 0.1080
Epoch 7/20
4342137/4342137 [============

4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2279 - accuracy: 0.0807 - val_loss: 3.2296 - val_accuracy: 0.0795
Epoch 15/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 3.2276 - accuracy: 0.0808 - val_loss: 3.2226 - val_accuracy: 0.0822
Epoch 16/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2265 - accuracy: 0.0808 - val_loss: 3.2288 - val_accuracy: 0.0796
Epoch 17/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2262 - accuracy: 0.0811 - val_loss: 3.2307 - val_accuracy: 0.0818
Epoch 18/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2262 - accuracy: 0.0811 - val_loss: 3.2194 - val_accuracy: 0.0819
Epoch 19/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2253 - accuracy: 0.0814 - val_loss: 3.2204 - val_accuracy: 0.0822
Epoch 20/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2254 - accurac